## Prerequisite:

In [267]:
import requests
from pymongo import MongoClient
api = 'http://api.nobelprize.org/v1/{}.json'

In [268]:
client = MongoClient('localhost',27017)
db = client['nobel_api']

In [269]:
words = ['prizes','laureates']

### Note: When we use 'insert_one' than it should be a single object or dict in terms of python, but when we use 'insert_many' then it should be an array or list in terms of Python.

In [270]:
for word in words:
    singular=word[:-1]
    response = requests.get(api.format(singular))
    result = response.json()[word]
    db[word].insert_many(result)

### Viewing collection in database:

In [271]:
db.collection_names(include_system_collections=False)

/home/khawajaosama/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prizes', 'laureates']

In [272]:
result

[{'_id': ObjectId('5c49663d41ddc1194e9507f7'),
  'born': '1845-03-27',
  'bornCity': 'Lennep (now Remscheid)',
  'bornCountry': 'Prussia (now Germany)',
  'bornCountryCode': 'DE',
  'died': '1923-02-10',
  'diedCity': 'Munich',
  'diedCountry': 'Germany',
  'diedCountryCode': 'DE',
  'firstname': 'Wilhelm Conrad',
  'gender': 'male',
  'id': '1',
  'prizes': [{'affiliations': [{'city': 'Munich',
      'country': 'Germany',
      'name': 'Munich University'}],
    'category': 'physics',
    'motivation': '"in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him"',
    'share': '1',
    'year': '1901'}],
  'surname': 'Röntgen'},
 {'_id': ObjectId('5c49663d41ddc1194e9507f8'),
  'born': '1853-07-18',
  'bornCity': 'Arnhem',
  'bornCountry': 'the Netherlands',
  'bornCountryCode': 'NL',
  'died': '1928-02-04',
  'diedCountry': 'the Netherlands',
  'diedCountryCode': 'NL',
  'firstname': 'Hendrik Antoon',
  'gender': '

In [273]:
n_prize = db.prizes.count_documents({})

n_laureates = db.laureates.count_documents({})

print(n_prize)

print(n_laureates)

590
934


### find_one method gives first document of a collection 

In [274]:
first_document = db.prizes.find_one({})
print(first_document)

{'_id': ObjectId('5c49663841ddc1194e9505a9'), 'year': '2018', 'laureates': [{'firstname': 'Arthur', 'surname': 'Ashkin', 'id': '960', 'share': '2', 'motivation': '"for the optical tweezers and their application to biological systems"'}, {'firstname': 'Gérard', 'surname': 'Mourou', 'id': '961', 'share': '4', 'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"'}, {'firstname': 'Donna', 'surname': 'Strickland', 'id': '962', 'share': '4', 'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"'}], 'overallMotivation': '“for groundbreaking inventions in the field of laser physics”', 'category': 'physics'}


### List of Database Names:

In [275]:
client.list_database_names()

['nobel_api', 'local', 'admin']

### List of Collection Names: 

In [276]:
client.nobel_api.list_collection_names()

['prizes', 'system.indexes', 'laureates']

## Filtering Some Documents:

In [277]:
print(db.laureates.count_documents({
  'gender': 'female'
})
)

print(db.laureates.count_documents({
  'diedCountry': 'France'
})
)

print(db.laureates.count_documents({
  'bornCity': 'Warsaw'
}))

print(db.laureates.count_documents({
  'gender': 'female',
  'diedCountry': 'France',
  'bornCity': 'Warsaw'
}))


51
50
2
1


## Query Operators:

In [278]:
print(db.laureates.count_documents({
    'diedCountry' : {
        '$in' : ['USA','France']           # count documents in USA or France
    }
}))

print(db.laureates.find_one({
    'diedCountry' : {
        '$in' : ['USA','France']         # show the result which find first
    }
}))

259
{'bornCity': 'Paris', 'prizes': [{'year': '1903', 'affiliations': [{'country': 'France', 'name': 'École Polytechnique', 'city': 'Paris'}], 'share': '2', 'motivation': '"in recognition of the extraordinary services he has rendered by his discovery of spontaneous radioactivity"', 'category': 'physics'}], '_id': ObjectId('5c49663d41ddc1194e9507fa'), 'firstname': 'Antoine Henri', 'gender': 'male', 'diedCountryCode': 'FR', 'surname': 'Becquerel', 'bornCountryCode': 'FR', 'diedCountry': 'France', 'died': '1908-08-25', 'bornCountry': 'France', 'id': '4', 'born': '1852-12-15'}


### Not Equal To...

In [279]:
print(db.laureates.count_documents({
 'diedCountry': {
  '$ne': 'France'              # not equals to
 }}))

884


### Greater than and less than equals to...

In [280]:
print(db.laureates.count_documents(
{
    'diedCountry' : {
        '$gt' : 'Belgium',
        '$lte' : 'USA'
    }
}
))

455


The born field in a laureate collection document records the date of birth of that laureate. born values are of the form "YYYY-MM-DD", also known as ISO 8601 format. An example value is "1937-02-01", for February 1st, 1937. This format is convenient for lexicographic comparison. For example, the query:

In [281]:
db.laureates.count_documents({"born": {"$lt": "1900-05-07"}})

327

#### Create a filter criteria to count (save as count) laureates who died ("diedCountry") in the USA ("USA").

In [282]:
# Create a filter for laureates who died in the USA
criteria = {'diedCountry': 'USA'}

# Save a count of these laureates
count = db.laureates.count_documents(criteria)
print(count)

209


#### Create a filter criteria to count laureates who died in the United States but were born ("bornCountry") in Germany.

In [283]:
# Create a filter for laureates who died in the USA but were born in Germany
criteria = {'diedCountry': 'USA', 'bornCountry': 'Germany'}

# Save a count
count = db.laureates.count_documents(criteria)
print(count)

5


#### Count laureates who died in the USA, were born in Germany, and whose first name ("firstname") was "Albert".

In [284]:
# Create a filter for Germany-born laureates who died in the USA and with the first name "Albert"
criteria = {'diedCountry': 'USA', 'bornCountry': 'Germany', 'firstname': 'Albert'}

# Save the count
count = db.laureates.count_documents(criteria)
print(count)

1


#### how many laureates were born in any of "Canada", "Mexico", or "USA"?

In [285]:
# Save a filter for laureates born in the USA, Canada, or Mexico
criteria = {'bornCountry': {"$in": ["Canada", "Mexico",  "USA"]}}

# Count them and save the count
count = db.laureates.count_documents(criteria)
print(count)

291


#### How many laureates died in the USA but were not born there? Save your filter as criteria and your count as count.

In [286]:
# Save a filter for laureates who died in the USA and were not born there
criteria = {'bornCountry': {"$ne": 'USA'}, 'diedCountry': 'USA'}

# Count them
count = db.laureates.count_documents(criteria)
print(count)

69


## Dot Notation: Reach Into Substructure:

In [287]:
db.laureates.find_one({
  "firstname": "Walter",
  "surname": "Kohn"})

{'_id': ObjectId('5c49663d41ddc1194e950913'),
 'born': '1923-03-09',
 'bornCity': 'Vienna',
 'bornCountry': 'Austria',
 'bornCountryCode': 'AT',
 'died': '2016-04-19',
 'diedCity': 'Santa Barbara, CA',
 'diedCountry': 'USA',
 'diedCountryCode': 'US',
 'firstname': 'Walter',
 'gender': 'male',
 'id': '290',
 'prizes': [{'affiliations': [{'city': 'Santa Barbara, CA',
     'country': 'USA',
     'name': 'University of California'}],
   'category': 'chemistry',
   'motivation': '"for his development of the density-functional theory"',
   'share': '2',
   'year': '1998'}],
 'surname': 'Kohn'}

In [288]:
db.laureates.count_documents({
    'prizes.affiliations.name' : 'University of California'
})

34

### Existing Of A field is found out by:

In [289]:
print(db.laureates.count_documents({}))
db.laureates.count_documents({'prizes':{'$exists':True}})   # yes it does

934


934

### To check the value of a field in collection:

In [290]:
db.laureates.count_documents({
    'prize.0': {
        '$exists':True
    }
})

0

#### Save a filter criteria for laureates born in (bornCountry) "Austria" with a non-Austria prizes.affiliations.country.

In [291]:
# Filter for laureates born in Austria with non-Austria prize affiliation
criteria = {'bornCountry': 'Austria', 'prizes.affiliations.country': {"$ne": 'Austria'}}

# Count the number of such laureates
count = db.laureates.count_documents(criteria)
print(count)

10


#### Use a filter document (criteria) to confirm that every document in the laureates collection has a "born" field.

In [292]:
# Filter for documents without a "born" field
criteria = {'born': {'$exists': False}}
assert db.laureates.count_documents(criteria) == 0

#### Use a filter document (criteria) to fetch a laureates document with at least three elements in its "prizes" array.

In [293]:
# Filter for laureates with at least three prizes
criteria = {'prizes.2': {'$exists': True}}
print(db.laureates.find_one(criteria))

{'died': '0000-00-00', 'prizes': [{'year': '1917', 'affiliations': [[]], 'share': '1', 'category': 'peace'}, {'year': '1944', 'affiliations': [[]], 'share': '1', 'category': 'peace'}, {'year': '1963', 'affiliations': [[]], 'share': '2', 'category': 'peace'}], '_id': ObjectId('5c49663d41ddc1194e9509d2'), 'firstname': 'Comité international de la Croix Rouge (International Committee of the Red Cross)', 'gender': 'org', 'id': '482', 'born': '0000-00-00'}
